Emma Finn

12/12/2023

Dr. Iams

Applied Math 111

# **Rader's Algorithim for the Discrete Fourier Transform**

# **ABSTRACT**
The purpose of this project was to understand Rader's Fast Fourier Transformation algorithim from two angles. The first approach was to explore the number theory behind the technique and understand how exactly a DFT can be expressed as a convolution and how the relatiosnhip to the group $\mathbb{Z}_{p}^{x}$ helps us to decompose the computation. The second approach was to implement Rader's Algorithim in Python, so that we could compute some examples and show that Rader's Algorithim is orders of magnitude better than a naive implementation. I developed a deeper understanding of groups, Fourier transformations, and the order of algorithims.


# **INTRODUCTION**
The Institute of Electrical and Electronics Engineers lists the FFT as one of the ten most important algorithims of the 20th century. [1] The FFT is uqbiquitous across disciplines from straigtforward applications to signal processing to use in convolutional neural networks. Rader's algorithim was the first of a class of FFT methods that allowed the decomposition of prime length sequences [3].



Broadly, the goal of Fourier Transform is to decompose a signal into a sum of periodic functions (here sines and cosines with different frequencies). Often mathemticians think of Fourier transformations as mappings of a continuous function from a  time domain to a frequency domain. Here, since computers work best with discrete data, we'll consider a fourier transformation as mapping a set of N sampled points from a time domain to a set of N points in a frequency domain.


This process of decomposition turns out to be extremly computationally intensive. A naive implementation requries on the order of $N^2$ multiplications alone [2]. For a tool used everywhere (radios, televisions, etc.), this level of inefficiency is impractical.


The simplest algorithims (e.g., Cooley-Tukey) are "divide and conquer" algorithims [2]. They work when N is a composite number. In particular, Cooley-Tukey subidvides N into its prime factors and cleverly sums their DFTs resulting in an algorithim on the order of $Nlog(N)$. However, when N is prime, no such divide and conquor method is possible. [2]


This is precisely the problem that Rader's algorithim solves. Instead of decomposing N into a product, it treats a fourier transformation as a (discrete) convolution, instead. The sequences convolved are carefully chosen and indexed to maximize computational efficiency. The indexing of these two sequences (which will be explored in much greater depth later) is chosen according to the group generator of the cyclic group $\mathbb{Z}_{p}^{x}$.


In this project, I detail the number theory that makes this possible and offer a few proofs for the key points that allow this. I also work through one small example by hand, illustrating for a small p that Rader's method results in the same output as the naive impelmetation.


I then move to the implementation of the algorithim, offering first a *very* naive fourier transformation. I compare that to built in methods. I then implement Rader's algorithim and its accompnaying helper functions. I show for a series of examples that it's both much faster than the naive implementation.


# **BACKGROUND**
The origin of my interest in this problem was in the number theory that makes this method possible. I had never studied number theory or group theory formally, so what follows is everything relevant that I learned for this project.

Let's start with a few definitons, whose usefulness I'll motivate after introducing them.
# Fourier Definitions


1.   Roots of Unity [4]: $$ \omega_N = e^{\frac{-2 \pi i}{N}} $$
2.   Fourier Transform [2]: $$ H_n = \sum_{k=0}^{N-1} h_k e^{\frac{2\pi ikn}{N}}$$ for $n \in \{ 0,1,2,...N\}$
  Is the general equation for a DFT, where N is the number of points sampled k is our indexing variable within the sum that produces each term and n is our indexing variable for the N different sums we compute, and i is the imaginary number.

# Convolution Definitions


1.   Discrete Convolution: $$\mathcal{F}\{g * h\}(n) = \sum_{k=-∞}^{∞} g[k]*h[n-k] $$ for $n \in \{ 0,1,2,...N\}$

2.   Convolution Theorem [5]: $$\mathcal{F}\{f * g\}(x) = \mathcal{F}(f(x))\bullet \mathcal{F}(g(x))$$ Where $\mathcal{F}$ denotes the discrete fourier transform, * denotes a convolution, $\bullet$ denotes pointwise multiplication, and f(x), g(x) represent sequences to be convolved rather than functions. The key idea here is that **"Convolution in the time domain is multiplication in the frequency domain."**


# Group Definitions
1.   **Group** a set G containing n elements and some arithmetic operation that is either one-to-one or many-to-one (we'll call this operation *) that satisfies the following properites:
      *   Closure: for any a,b ∈ G, then if c = a*b, c ∈ G
      *   Associativity: (a * b) * c = a * (b * c) $ for a,b,c ∈ G
      *   Identity: ∃e ∈G such that a * e = a ∀a∈G
      *   Inverse: for every a ∈ G  ∃b such that a * b = e

2.   **Abelian Group** a communative group -- ie it satisfies all of the above and ∀ a,b ∈ G,  a * b = b * a

3.   **Order** (for a finite group) *the number of elements in that group* or (for an elment of a group) *the smallest m such that $a^m = e$ where e is the identity element of the group* (this is connected to the first definition because this is another way of writing the number of elements of the subgroup generated by that element)
4.   **Period** Least common multiple of the orders of the lements of the group


(Definitions from Garg, pg 11, SIAM paper, Shlomo pg 672)

It's by no means obvious why any of these definitions are relevant to a question about Fourier transformations. In order to motivate our choice of technique, I'm going to show the example p = 7, then break down the steps and include any relevant theorems to show that this is true in the general case.


# Motivating Example

Let **h** = [ $h_1, h_2, h_3, h_4, h_5, h_6, h_7$ ] be 7 points sampled from some function on which we want to perform a fourier transformation.




To recap, the tools needed to solve this problem were:


1.   Definition and Properties of Group and Multiplicative Inverses
1.   $\mathbb{Z}_{p}^{x}$ is Cyclic with Generator g
2.   Definition and Properties of Roots of Unity
1.   Definition of a Fourier Transformation
2.   Definition of a Convolution
2.   Convolution Theorem

Describe the necessary mathematics, or other information, that you needed to know or learn to make progress on this project.

Include citations, as appropriate.

**METHODS**


# **METHODS**
Describe numerical methods being used in your project.  Share what you learned about the method and summarize any work you did to implement it, and include implementation code.

In [1]:
import numpy as np
import sympy as sy
import math
from scipy import linalg, fft

**Import Example Signal**

In [2]:
#Import Example Signal to Decompose

**BUILT-IN FFT**

In [3]:
#Built In Method
def builtDFT(x):
  return np.fft.fft(x)

**NAIVE FFT**

In [4]:
#Intuitive Implementation
#X_m = sum from k = 0 to N-1 of e^(-2pi*jkm/N)x_k for m {0,1,2,.. N-1} where j is i for some reason
def DFT0(x):
  N = len(x)
  x = np.asarray(x, dtype=float)
  Y = np.zeros((N,1), dtype=complex)
  for m in range(N):
    Y[m] = np.sum([x[k]*np.exp(-2j*np.pi*k*m/N) for k in range(N)])
  return Y

t = np.linspace(0,50,1000)
y = np.sin(t)
#x = np.random.random(100)

# transpose is to make the shapes match
DFT_built_in = np.asarray([np.fft.fft(y)]).T
DFT_naive = DFT0(y)

#Hey look it works!!
np.allclose(DFT_naive,DFT_built_in)

True

**RADER'S FFT**

In [5]:
#Helper Function 1 --> Finding the Group Generator
def findGenerator(n):
    phi = n - 1
    prime_factors = sy.primefactors(phi)

    for i in range(2, n):
        is_generator = True
        for j in prime_factors:
            if pow(i, phi // j, n) == 1:
                is_generator = False
                break
        if is_generator:
            return i
    return -1

In [6]:
# Helper Function 2 --> Reordering  input data AND reordering output of the DFT when g = g_inv
def rearrange(input, g, p):
  # a_q = h_(g^q), g is group generator, p is prime length
  n = len(input)
  index = [0] * n
  for q in range(n):
    index[q] = pow(g, q, p) - 1
  re = [0] * n
  for q in range(n):
    re[int(index[q])] = input[q]
  # rearrange values in input array
  for q in range(n):
    input[q] = re[q]
  return input

In [7]:
# Helper Function 3 --> Arranging powers of omega
def omega_sequence(g_inv, p):
  # g_inv is inverse of group generator, p is prime length
  n = p - 1
  exponent = [0] * n
  w = np.exp(-2j*np.pi/p)
  for q in range(n):
    exponent[q] = pow(g_inv, q, p)
  omega = w**exponent
  return omega

In [8]:
# Check that input length is prime, if not, return error
def raders_fft(input):
  N = len(input)
  if not sy.isprime(N):
    print("error")

  # Seperate the zero-indexed part of the calculation
  y0 = input[0]
  input_1 = np.delete(input, 0)

  # Find the group generator --> write helper function that uses euler totient to find primitive roots
  g = findGenerator(N)

  # Find the inverse group generator
  g_inv = sy.mod_inverse(g,N)

  # Rewrite the sum of the non-zero indexed terms, reordering and reindexing according to the cyclic properties of the group
  a_q = rearrange(input_1, g, N)
  b_q = omega_sequence(g_inv, N)

  # Compute the pointwise multiplication of the dtf of both sequences above F(a*b) = AB --> (a*b) = F_inv(AB) (convolution theorem)
  A_q = np.fft.fft(a_q)
  B_q = np.fft.fft(b_q)
  prod = A_q * B_q

  # Take the inverse DFT of their product
  inverse_DFT = fft.ifft(prod)

  # Reorder using SAME rearrange function but with g_inv instead of g as inverse
  output_less_y0 = rearrange(inverse_DFT, g_inv, p)

  # Add back the zero term
  output = output_less_y0 + y0

  # Generate initial term by summing input terms
  Y_0 = np.sum(input)

  # Prepend first term to sequence
  output = np.insert(output, 0, Y_0)

  return output

# **RESULTS**
Describe the results or accomplishments of your project.  Include figures (tables, plots, etc, with their generating code), written captions describing the information in the figure, and text explaining the results of your project work.

In [9]:
# Does Rader's Algorithim Work?
x = np.random.random(1171)
real = np.fft.fft(x)
raders = raders_fft(x)
np.allclose(DFT_naive,DFT_built_in)

NameError: name 'p' is not defined

In [ ]:
# NAIVE FFT VS RADER'S VS BUILT IN
# Let's see how long it takes
%timeit DFT0(x)
%timeit raders_fft(x)
%timeit np.fft.fft(x)

2.67 s ± 656 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.65 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
120 µs ± 29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#**DISCUSSIONS/CONCLUSIONS**
# Discussion of Results
Discuss any results, and what you learned from the project work.

# Revisions and Extensions
If I had the chance to do this project over again, I'd change the language I programmed in. While Python is the language I'm most comfortable with, I now  understand why I didn't find anyone else implementing Rader's method (or any DFT designed for efficiency) using it. While the code is readable and very closely related to the computations I performed, it's not the most efficient. While I'm very glad I was able to get within two orders of magnitude of the built-in method, I think I would have been able to do much better using a language designed for efficiency where more vectorization is possible. I'd also spend more time optimizing my helper functions, which I think are slowing down the process quite a bit. I wish I had a chance to implement zero-padding to the nearest power of two and compare that directly to the pure Rader FFT. In that scenario, I'd also have ideally liked to write my own Cooley-Tukey method, so that I could avoid using any built ins, and compute Rader's Method with Zero Padding using only my own code. It would have been really interesting to compare that to the built in method to see how much my helper functions are slowing me down.

# Lingering Questions
Since this took almost exactly as much time as I expected, there's nothing that I intended to do that hasn't worked yet. However, the two things that I mentioned in my proposal that I didn't think I'd have time to implement (and didn't) but still want to explore more are Bluestein's method and proving analytically that when zero-padded optimally, Rader's Algorithim is $O(NlogN)$.I'd also work a little more on more modern implementations of FFTs, especially the Chirp-Z transform and learn more about non-fourier time frequency transformations, like the Laplace transform. In some of the litterature I've read, they mentioned a hardware implementation of the FFT, which I'd love to learn more about [6]. I still don't think I really understand in general how we go from a physical signal to something a computer takes in to a data set Rader's algorithim acts upon.

  

**BIBLIOGRAPHY**

[1] https://www.computer.org/csdl/magazine/cs/2000/01/c1022/13rRUxBJhBm

[2] Elementary Number Theory and Rader’s FFT: 10.1137/15M1044990

[3] https://ietresearch.onlinelibrary.wiley.com/doi/full/10.1049/iet-cds.2018.5200#:~:text=Rader%20algorithm%20was%20introduced%20as,the%20periodicity%20of%20the%20DFT

[4]https://cs.stanford.edu/~rayyli/static/contest/lectures/Ray%20Li%20rootsofunity.pdf

[5] https://betterexplained.com/articles/intuitive-convolution/

[6] https://web.mit.edu/6.111/www/f2017/handouts/FFTtutorial121102.pdf

